Let's try to combine MultiArmBandit UCB into a Gaussian Process Regressor kernel.

In [9]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

class CoCaBO:
    def __init__(self, continuous_dim, categorical_names, kernel=None, noise_var=1e-5):
        """
        CoCaBO algorithm initialization
        :param continuous_dim: Dimensionality of continuous variables
        :param categorical_names: List of categorical names (e.g., ['omic1', 'omic2', ..., 'omic5'])
        :param kernel: Kernel for the Gaussian Process (Matern kernel by default)
        :param noise_var: Noise variance for Gaussian Process
        """
        self.continuous_dim = continuous_dim
        self.categorical_names = categorical_names
        self.categorical_dim = len(categorical_names)  # Number of categorical arms
        
        # Initialize the kernel for Gaussian Process (Matern kernel)
        self.kernel = kernel if kernel else Matern(length_scale=1.0, nu=1.5)  # Matern kernel with nu=1.5
        
        # Initialize the Gaussian Process Regressor
        self.gpr = GaussianProcessRegressor(kernel=self.kernel, alpha=noise_var)
        
        # Storage for past data
        self.X_cont = []  # Stores continuous variables
        self.X_cat = []   # Stores categorical variables (as indices)
        self.y = []       # Stores corresponding objective function values
        
        # Initialize Multi-Armed Bandit (MAB) for categorical variable
        self.mab_rewards = np.zeros(self.categorical_dim)  # Rewards for each categorical arm
        self.mab_counts = np.zeros(self.categorical_dim)   # Count of pulls for each categorical arm
    
    def fit(self, X_cont, X_cat, y):
        """
        Fit the Gaussian Process model on both continuous and categorical data.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :param y: Objective function values (rewards)
        """
        # Combine continuous and categorical data
        X_combined = np.hstack((X_cont, X_cat))
        
        # Fit the Gaussian Process Regressor model
        self.gpr.fit(X_combined, y)
        
        # Store the data for future optimization
        self.X_cont.extend(X_cont)
        self.X_cat.extend(X_cat)
        self.y.extend(y)
    
    def predict(self, X_cont, X_cat):
        """
        Predict mean and variance for new points.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :return: Mean and standard deviation from GP
        """
        X_combined = np.hstack((X_cont, X_cat))
        mean, std = self.gpr.predict(X_combined, return_std=True)
        return mean, std
    
    def ucb(self, X_cont, X_cat, kappa=2.0):
        """
        Upper Confidence Bound (UCB) acquisition function for continuous variables.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :param kappa: Exploration parameter for UCB
        :return: UCB values
        """
        mean, std = self.predict(X_cont, X_cat)
        ucb_values = mean + kappa * std
        return ucb_values
    
    def select_categorical_arm(self):
        """
        Select the categorical arm using the best-performing arm (exploitation).
        :return: Best categorical arm based on historical rewards
        """
        # Select the categorical arm with the highest average reward
        best_arm_idx = np.argmax(self.mab_rewards)
        return self.categorical_names[best_arm_idx]  # Return the string name of the selected arm
    
    def update_mab(self, arm_idx, reward):
        """
        Update the reward distribution of the Multi-Armed Bandit (MAB).
        :param arm_idx: The index of the arm that was pulled
        :param reward: The reward received for pulling the arm
        """
        self.mab_counts[arm_idx] += 1
        # Update the reward for the selected arm (simple average reward)
        self.mab_rewards[arm_idx] = ((self.mab_counts[arm_idx] - 1) * self.mab_rewards[arm_idx] + reward) / self.mab_counts[arm_idx]
    
    def optimize(self, X_cont, X_cat, kappa=2.0):
        """
        Optimize the acquisition function (UCB) and select the best continuous and categorical values.
        :param X_cont: New continuous points to evaluate
        :param X_cat: New categorical points
        :param kappa: Exploration parameter for UCB
        :return: Best continuous and categorical values
        """
        # Select categorical arm using MAB (best-performing arm based on reward distribution)
        h_t = self.select_categorical_arm()
        # Convert the categorical arm name to its index
        h_t_idx = self.categorical_names.index(h_t)
        
        # Optimize continuous variable using UCB
        ucb_values = self.ucb(X_cont, np.full((X_cont.shape[0], 1), h_t_idx), kappa)
        best_idx = np.argmax(ucb_values)  # Select the index with the highest UCB value
        
        # Return the best continuous and categorical pair (return categorical arm as string)
        return X_cont[best_idx], h_t

# Example Usage

# Categorical names as strings
categorical_names = ['omic1', 'omic2', 'omic3', 'omic4', 'omic5']

# Example continuous and categorical variables (encoded as indices for MAB)
X_cont = np.array([[0.5], [0.2], [0.7]])  # Example continuous variables
X_cat = np.array([[0], [1], [2]])  # Example categorical variables (as indices)
y = np.array([0.3, 0.7, 0.5])  # Objective values (rewards)

# Instantiate the CoCaBO object with categorical names and Matern kernel
optimizer = CoCaBO(continuous_dim=1, categorical_names=categorical_names)

# Fit the model to the data
optimizer.fit(X_cont, X_cat, y)

# Predict UCB values for new points
new_cont = np.array([[0.6], [0.3]])  # New continuous points to evaluate
new_cat = np.array([[1], [0]])  # New categorical points (encoded as indices)

# Optimize to get the best combination of continuous and categorical variables
best_cont, best_cat = optimizer.optimize(new_cont, new_cat)
print(f"Best continuous: {best_cont}, Best categorical: {best_cat}")

# Update MAB and fit with new data (simulating the query)
best_cat_idx = categorical_names.index(best_cat)  # Convert the name back to an index
optimizer.update_mab(best_cat_idx, reward=0.8)  # Update with new reward (simulated)


Best continuous: [0.3], Best categorical: omic1


This is something else, the last version I created.... I'm not really sure at this point but I need to go through it all cause the math is completely wrong at the moment.

In [13]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

class CoCaBO:
    def __init__(self, continuous_dim, categorical_names, kernel=None, noise_var=1e-5):
        """
        CoCaBO algorithm initialization
        :param continuous_dim: Dimensionality of continuous variables
        :param categorical_names: List of categorical names (e.g., ['omic1', 'omic2', ..., 'omic5'])
        :param kernel: Kernel for the Gaussian Process (Matern kernel by default)
        :param noise_var: Noise variance for Gaussian Process
        """
        self.continuous_dim = continuous_dim
        self.categorical_names = categorical_names
        self.categorical_dim = len(categorical_names)  # Number of categorical arms
        
        # Initialize the kernel for Gaussian Process (Matern kernel)
        self.kernel = kernel if kernel else Matern(length_scale=1.0, nu=1.5)  # Matern kernel with nu=1.5
        
        # Initialize the Gaussian Process Regressor
        self.gpr = GaussianProcessRegressor(kernel=self.kernel, alpha=noise_var)
        
        # Storage for past data
        self.X_cont = []  # Stores continuous variables
        self.X_cat = []   # Stores categorical variables (as indices)
        self.y = []       # Stores corresponding objective function values
        
        # Initialize Multi-Armed Bandit (MAB) for categorical variable
        self.mab_rewards = np.zeros(self.categorical_dim)  # Rewards for each categorical arm
        self.mab_counts = np.zeros(self.categorical_dim)   # Count of pulls for each categorical arm
    
    def fit(self, X_cont, X_cat, y):
        """
        Fit the Gaussian Process model on both continuous and categorical data.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :param y: Objective function values (rewards)
        """
        # Combine continuous and categorical data
        X_combined = np.hstack((X_cont, X_cat))
        
        # Fit the Gaussian Process Regressor model
        self.gpr.fit(X_combined, y)
        
        # Store the data for future optimization
        self.X_cont.extend(X_cont)
        self.X_cat.extend(X_cat)
        self.y.extend(y)
    
    def predict(self, X_cont, X_cat):
        """
        Predict mean and variance for new points.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :return: Mean and standard deviation from GP
        """
        X_combined = np.hstack((X_cont, X_cat))
        mean, std = self.gpr.predict(X_combined, return_std=True)
        return mean, std
    
    def ucb(self, X_cont, X_cat, kappa=2.0):
        """
        Upper Confidence Bound (UCB) acquisition function for continuous variables.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :param kappa: Exploration parameter for UCB
        :return: UCB values
        """
        mean, std = self.predict(X_cont, X_cat)
        ucb_values = mean + kappa * std
        return ucb_values
    
    def select_categorical_arm(self):
        """
        Select the categorical arm using the best-performing arm (exploitation).
        :return: Best categorical arm based on historical rewards
        """
        # Select the categorical arm with the highest average reward
        best_arm_idx = np.argmax(self.mab_rewards)
        return self.categorical_names[best_arm_idx]  # Return the string name of the selected arm
    
    def update_mab(self, arm_idx, reward):
        """
        Update the reward distribution of the Multi-Armed Bandit (MAB).
        :param arm_idx: The index of the arm that was pulled
        :param reward: The reward received for pulling the arm
        """
        self.mab_counts[arm_idx] += 1
        # Update the reward for the selected arm (simple average reward)
        self.mab_rewards[arm_idx] = ((self.mab_counts[arm_idx] - 1) * self.mab_rewards[arm_idx] + reward) / self.mab_counts[arm_idx]
    
    def optimize(self, X_cont, X_cat, kappa=2.0):
        """
        Optimize the acquisition function (UCB) and select the best continuous and categorical values.
        :param X_cont: New continuous points to evaluate
        :param X_cat: New categorical points
        :param kappa: Exploration parameter for UCB
        :return: Best continuous and categorical values
        """
        # Select categorical arm using MAB (best-performing arm based on reward distribution)
        h_t = self.select_categorical_arm()
        # Convert the categorical arm name to its index
        h_t_idx = self.categorical_names.index(h_t)
        
        # Optimize continuous variable using UCB
        ucb_values = self.ucb(X_cont, np.full((X_cont.shape[0], 1), h_t_idx), kappa)
        best_idx = np.argmax(ucb_values)  # Select the index with the highest UCB value
        
        # Return the best continuous and categorical pair (return categorical arm as string)
        return X_cont[best_idx], h_t


# Example Usage

# Categorical names as strings
categorical_names = ['omics1', 'omic2', 'omic2', 'omic4', 'omic5']

# Generate a larger dataset (1000 points) for continuous and categorical variables
X_cont = np.random.uniform(0, 1, (1000, 1))  # 1000 random continuous points in range [0, 1]
X_cat = np.random.randint(0, 5, (1000, 1))  # 1000 random categorical points (encoded as indices)
y = np.random.uniform(0, 1, 1000)  # Random objective function values between 0 and 1

# Instantiate the CoCaBO object with categorical names and Matern kernel
optimizer = CoCaBO(continuous_dim=1, categorical_names=categorical_names)

# Fit the model to the larger dataset
optimizer.fit(X_cont, X_cat, y)

# Predict UCB values for new points (new continuous and categorical combinations)
new_cont = np.random.uniform(0, 1, (5, 1))  # 5 new random continuous points to evaluate
new_cat = np.random.randint(0, 5, (5, 1))  # 5 new random categorical points (encoded as indices)

# Optimize to get the best combination of continuous and categorical variables
for i in range(5):  # Simulate 5 predictions
    best_cont, best_cat = optimizer.optimize(new_cont, new_cat)
    print(f"Prediction {i+1}: Best continuous: {best_cont}, Best categorical: {best_cat}")

    # Simulate a reward for this prediction and update the MAB and GP model
    best_cat_idx = categorical_names.index(best_cat)  # Convert the name back to an index
    reward = np.random.uniform(0, 1)  # Random reward between 0 and 1
    optimizer.update_mab(best_cat_idx, reward)  # Update the model with new reward


Prediction 1: Best continuous: [0.78331689], Best categorical: omics1
Prediction 2: Best continuous: [0.78331689], Best categorical: omics1
Prediction 3: Best continuous: [0.78331689], Best categorical: omics1
Prediction 4: Best continuous: [0.78331689], Best categorical: omics1
Prediction 5: Best continuous: [0.78331689], Best categorical: omics1


This needs fixing, but the below is attempting to use the liar's approach to use sequential predictions to generate a batch result.

In [4]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

class CoCaBO:
    def __init__(self, continuous_dim, categorical_names, kernel=None, noise_var=1e-5):
        """
        CoCaBO algorithm initialization
        :param continuous_dim: Dimensionality of continuous variables
        :param categorical_names: List of categorical names (e.g., ['omic1', 'omic2', ..., 'omic5'])
        :param kernel: Kernel for the Gaussian Process (Matern kernel by default)
        :param noise_var: Noise variance for Gaussian Process
        """
        self.continuous_dim = continuous_dim
        self.categorical_names = categorical_names
        self.categorical_dim = len(categorical_names)  # Number of categorical arms
        
        # Initialize the kernel for Gaussian Process (Matern kernel)
        self.kernel = kernel if kernel else Matern(length_scale=1.0, nu=1.5)  # Matern kernel with nu=1.5
        
        # Initialize the Gaussian Process Regressor
        self.gpr = GaussianProcessRegressor(kernel=self.kernel, alpha=noise_var)
        
        # Storage for past data
        self.X_cont = []  # Stores continuous variables
        self.X_cat = []   # Stores categorical variables (as indices)
        self.y = []       # Stores corresponding objective function values
        
        # Initialize Multi-Armed Bandit (MAB) for categorical variable
        self.mab_rewards = np.zeros(self.categorical_dim)  # Rewards for each categorical arm
        self.mab_counts = np.zeros(self.categorical_dim)   # Count of pulls for each categorical arm
    
    def fit(self, X_cont, X_cat, y):
        """
        Fit the Gaussian Process model on both continuous and categorical data.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :param y: Objective function values (rewards)
        """
        # Combine continuous and categorical data
        X_combined = np.hstack((X_cont, X_cat))
        
        # Fit the Gaussian Process Regressor model
        self.gpr.fit(X_combined, y)
        
        # Store the data for future optimization
        self.X_cont.extend(X_cont)
        self.X_cat.extend(X_cat)
        self.y.extend(y)
    
    def predict(self, X_cont, X_cat):
        """
        Predict mean and variance for new points.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :return: Mean and standard deviation from GP
        """
        X_combined = np.hstack((X_cont, X_cat))
        mean, std = self.gpr.predict(X_combined, return_std=True)
        return mean, std
    
    def ucb(self, X_cont, X_cat, kappa=2.0):
        """
        Upper Confidence Bound (UCB) acquisition function for continuous variables.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :param kappa: Exploration parameter for UCB
        :return: UCB values
        """
        mean, std = self.predict(X_cont, X_cat)
        ucb_values = mean + kappa * std
        return ucb_values
    
    def select_categorical_arm(self):
        """
        Select the categorical arm using the best-performing arm (exploitation).
        :return: Best categorical arm based on historical rewards
        """
        # Select the categorical arm with the highest average reward
        best_arm_idx = np.argmax(self.mab_rewards)
        return self.categorical_names[best_arm_idx]  # Return the string name of the selected arm
    
    def update_mab(self, arm_idx, reward):
        """
        Update the reward distribution of the Multi-Armed Bandit (MAB).
        :param arm_idx: The index of the arm that was pulled
        :param reward: The reward received for pulling the arm
        """
        self.mab_counts[arm_idx] += 1
        # Update the reward for the selected arm (simple average reward)
        self.mab_rewards[arm_idx] = ((self.mab_counts[arm_idx] - 1) * self.mab_rewards[arm_idx] + reward) / self.mab_counts[arm_idx]
    
    def constant_liars_algorithm(self, X_cont, X_cat, n_predictions=5, kappa=2.0):
        """
        Constant Liar's Algorithm - Make multiple predictions using the GP and select the best-performing ones.
        :param X_cont: New continuous points to evaluate
        :param X_cat: New categorical points
        :param n_predictions: Number of predictions to make
        :param kappa: Exploration parameter for UCB
        :return: Best continuous and categorical values from the predictions
        """
        best_predictions = []
        
        for i in range(n_predictions):
            # Select categorical arm using MAB (best-performing arm based on reward distribution)
            h_t = self.select_categorical_arm()
            h_t_idx = self.categorical_names.index(h_t)  # Convert the arm name to its index
            
            # Predict UCB values for continuous and categorical variables
            ucb_values = self.ucb(X_cont, np.full((X_cont.shape[0], 1), h_t_idx), kappa)
            
            best_idx = np.argmax(ucb_values)  # Select the index with the highest UCB value
            best_cont = X_cont[best_idx]
            best_cat = h_t
            
            # Append prediction results (continuous, categorical)
            best_predictions.append((best_cont, best_cat, ucb_values[best_idx]))
            
            # Simulate querying the function and getting a new reward for the selected arm
            ft = np.random.normal(loc=0.5, scale=0.2)  # Simulated reward
            
            # Update MAB and GP model with new data
            self.update_mab(h_t_idx, ft)  # Update the MAB with the new reward
            self.fit(np.array([best_cont]), np.array([[h_t_idx]]), np.array([ft]))  # Update the GP model
            
            print(f"Prediction {i+1}: Best continuous: {best_cont}, Best categorical: {best_cat}, UCB value: {ucb_values[best_idx]}, Reward: {ft}")
        
        # Print all predictions with corresponding UCB values
        print("\nPredictions with UCB values:")
        for cont, cat, ucb_value in best_predictions:
            print(f"Best continuous: {cont}, Best categorical: {cat}, UCB value: {ucb_value}")
        
        # Return the best prediction (highest UCB)
        return max(best_predictions, key=lambda x: x[2])[:2]

    def optimize(self, X_cont, X_cat, n_predictions=5, kappa=2.0):
        """
        Optimize the acquisition function (UCB) and select the best continuous and categorical values.
        :param X_cont: New continuous points to evaluate
        :param X_cat: New categorical points
        :param n_predictions: Number of predictions to make using constant liar's algorithm
        :param kappa: Exploration parameter for UCB
        :return: Best continuous and categorical values
        """
        # Apply the constant liar's algorithm to make n predictions
        best_cont, best_cat = self.constant_liars_algorithm(X_cont, X_cat, n_predictions, kappa)
        
        # Return the best continuous and categorical pair
        return best_cont, best_cat

# Example Usage

np.random.seed(42)  # For reproducibility of results

# Categorical names as strings
categorical_names = ['omic1', 'omic2', 'omic3', 'omic4', 'omic5']

# Example continuous and categorical variables (encoded as indices for MAB)
X_cont = np.array([[0.5], [0.2], [0.7]])  # Example continuous variables
X_cat = np.array([[0], [1], [2]])  # Example categorical variables (as indices)
y = np.array([0.3, 0.7, 0.5])  # Objective values (rewards)

# Instantiate the CoCaBO object with categorical names and Matern kernel
optimizer = CoCaBO(continuous_dim=1, categorical_names=categorical_names)

# Fit the model to the data
optimizer.fit(X_cont, X_cat, y)

# Predict UCB values for new points
new_cont = np.array([[0.6], [0.3]])  # New continuous points to evaluate
new_cat = np.array([[1], [0]])  # New categorical points (encoded as indices)

# Optimize to get the best combination of continuous and categorical variables
best_cont, best_cat = optimizer.optimize(new_cont, new_cat, n_predictions=5)
print(f"\nBest continuous: {best_cont}, Best categorical: {best_cat}")


Prediction 1: Best continuous: [0.3], Best categorical: omic1, UCB value: 0.5025013626390963, Reward: 0.5993428306022466
Prediction 2: Best continuous: [0.6], Best categorical: omic1, UCB value: 1.3976455652602264, Reward: 0.4723471397657631
Prediction 3: Best continuous: [0.3], Best categorical: omic1, UCB value: 1.2828692572957117, Reward: 0.6295377076201385
Prediction 4: Best continuous: [0.6], Best categorical: omic1, UCB value: 1.4249351250927602, Reward: 0.8046059712816052
Prediction 5: Best continuous: [0.3], Best categorical: omic1, UCB value: 1.5831585168666864, Reward: 0.4531693250553328

Predictions with UCB values:
Best continuous: [0.3], Best categorical: omic1, UCB value: 0.5025013626390963
Best continuous: [0.6], Best categorical: omic1, UCB value: 1.3976455652602264
Best continuous: [0.3], Best categorical: omic1, UCB value: 1.2828692572957117
Best continuous: [0.6], Best categorical: omic1, UCB value: 1.4249351250927602
Best continuous: [0.3], Best categorical: omic1, 

Some softmax based method

In [5]:
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern

class CoCaBO:
    def __init__(self, continuous_dim, categorical_names, kernel=None, noise_var=1e-5, tau=1.0):
        """
        CoCaBO algorithm initialization
        :param continuous_dim: Dimensionality of continuous variables
        :param categorical_names: List of categorical names (e.g., ['omic1', 'omic2', ..., 'omic5'])
        :param kernel: Kernel for the Gaussian Process (Matern kernel by default)
        :param noise_var: Noise variance for Gaussian Process
        :param tau: Softmax temperature parameter (controls exploration-exploitation)
        """
        self.continuous_dim = continuous_dim
        self.categorical_names = categorical_names
        self.categorical_dim = len(categorical_names)  # Number of categorical arms
        
        # Initialize the kernel for Gaussian Process (Matern kernel)
        self.kernel = kernel if kernel else Matern(length_scale=1.0, nu=1.5)  # Matern kernel with nu=1.5
        
        # Initialize the Gaussian Process Regressor
        self.gpr = GaussianProcessRegressor(kernel=self.kernel, alpha=noise_var)
        
        # Storage for past data
        self.X_cont = []  # Stores continuous variables
        self.X_cat = []   # Stores categorical variables (as indices)
        self.y = []       # Stores corresponding objective function values
        
        # Initialize Multi-Armed Bandit (MAB) for categorical variable
        self.mab_rewards = np.zeros(self.categorical_dim)  # Rewards for each categorical arm
        self.mab_counts = np.zeros(self.categorical_dim)   # Count of pulls for each categorical arm
        self.tau = tau  # Softmax temperature for exploration-exploitation trade-off
    
    def fit(self, X_cont, X_cat, y):
        """
        Fit the Gaussian Process model on both continuous and categorical data.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :param y: Objective function values (rewards)
        """
        # Combine continuous and categorical data
        X_combined = np.hstack((X_cont, X_cat))
        
        # Fit the Gaussian Process Regressor model
        self.gpr.fit(X_combined, y)
        
        # Store the data for future optimization
        self.X_cont.extend(X_cont)
        self.X_cat.extend(X_cat)
        self.y.extend(y)
    
    def predict(self, X_cont, X_cat):
        """
        Predict mean and variance for new points.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :return: Mean and standard deviation from GP
        """
        X_combined = np.hstack((X_cont, X_cat))
        mean, std = self.gpr.predict(X_combined, return_std=True)
        return mean, std
    
    def ucb(self, X_cont, X_cat, kappa=2.0):
        """
        Upper Confidence Bound (UCB) acquisition function for continuous variables.
        :param X_cont: Continuous variables
        :param X_cat: Categorical variables (as indices)
        :param kappa: Exploration parameter for UCB
        :return: UCB values
        """
        mean, std = self.predict(X_cont, X_cat)
        ucb_values = mean + kappa * std
        return ucb_values
    
    def softmax_arm_selection(self):
        """
        Select the categorical arm using softmax-based exploration-exploitation.
        :return: Best categorical arm based on softmax selection
        """
        # Compute softmax probabilities for each arm based on rewards
        exp_rewards = np.exp(self.mab_rewards / self.tau)
        softmax_probs = exp_rewards / np.sum(exp_rewards)
        
        # Sample an arm based on softmax probabilities (exploration-exploitation balance)
        arm_idx = np.random.choice(self.categorical_dim, p=softmax_probs)
        return self.categorical_names[arm_idx]  # Return the string name of the selected arm
    
    def update_mab(self, arm_idx, reward):
        """
        Update the reward distribution of the Multi-Armed Bandit (MAB).
        :param arm_idx: The index of the arm that was pulled
        :param reward: The reward received for pulling the arm
        """
        self.mab_counts[arm_idx] += 1
        # Update the reward for the selected arm (simple average reward)
        self.mab_rewards[arm_idx] = ((self.mab_counts[arm_idx] - 1) * self.mab_rewards[arm_idx] + reward) / self.mab_counts[arm_idx]
    
    def constant_liars_algorithm(self, X_cont, X_cat, n_predictions=5, kappa=2.0):
        """
        Constant Liar's Algorithm - Make multiple predictions using the GP and select the best-performing ones.
        :param X_cont: New continuous points to evaluate
        :param X_cat: New categorical points
        :param n_predictions: Number of predictions to make
        :param kappa: Exploration parameter for UCB
        :return: Best continuous and categorical values from the predictions
        """
        best_predictions = []
        
        for i in range(n_predictions):
            # Select categorical arm using softmax (exploration-exploitation balance)
            h_t = self.softmax_arm_selection()
            h_t_idx = self.categorical_names.index(h_t)  # Convert the arm name to its index
            
            # Predict UCB values for continuous and categorical variables
            ucb_values = self.ucb(X_cont, np.full((X_cont.shape[0], 1), h_t_idx), kappa)
            
            best_idx = np.argmax(ucb_values)  # Select the index with the highest UCB value
            best_cont = X_cont[best_idx]
            best_cat = h_t
            
            # Append prediction results (continuous, categorical)
            best_predictions.append((best_cont, best_cat, ucb_values[best_idx]))
            
            # Simulate querying the function and getting a new reward for the selected arm
            ft = np.random.normal(loc=0.5, scale=0.2)  # Simulated reward
            
            # Update MAB and GP model with new data
            self.update_mab(h_t_idx, ft)  # Update the MAB with the new reward
            self.fit(np.array([best_cont]), np.array([[h_t_idx]]), np.array([ft]))  # Update the GP model
            
            print(f"Prediction {i+1}: Best continuous: {best_cont}, Best categorical: {best_cat}, UCB value: {ucb_values[best_idx]}, Reward: {ft}")
        
        # Print all predictions with corresponding UCB values
        print("\nPredictions with UCB values:")
        for cont, cat, ucb_value in best_predictions:
            print(f"Best continuous: {cont}, Best categorical: {cat}, UCB value: {ucb_value}")
        
        # Return the best prediction (highest UCB)
        return max(best_predictions, key=lambda x: x[2])[:2]

    def optimize(self, X_cont, X_cat, n_predictions=5, kappa=2.0):
        """
        Optimize the acquisition function (UCB) and select the best continuous and categorical values.
        :param X_cont: New continuous points to evaluate
        :param X_cat: New categorical points
        :param n_predictions: Number of predictions to make using constant liar's algorithm
        :param kappa: Exploration parameter for UCB
        :return: Best continuous and categorical values
        """
        # Apply the constant liar's algorithm to make n predictions
        best_cont, best_cat = self.constant_liars_algorithm(X_cont, X_cat, n_predictions, kappa)
        
        # Return the best continuous and categorical pair
        return best_cont, best_cat

# Example Usage

np.random.seed(42)  # For reproducibility of results

# Categorical names as strings
categorical_names = ['omic1', 'omic2', 'omic3', 'omic4', 'omic5']

# Example continuous and categorical variables (encoded as indices for MAB)
X_cont = np.array([[0.5], [0.2], [0.7]])  # Example continuous variables
X_cat = np.array([[0], [1], [2]])  # Example categorical variables (as indices)
y = np.array([0.3, 0.7, 0.5])  # Objective values (rewards)

# Instantiate the CoCaBO object with categorical names and Matern kernel
optimizer = CoCaBO(continuous_dim=1, categorical_names=categorical_names)

# Fit the model to the data
optimizer.fit(X_cont, X_cat, y)

# Predict UCB values for new points
new_cont = np.array([[0.6], [0.3]])  # New continuous points to evaluate
new_cat = np.array([[1], [0]])  # New categorical points (encoded as indices)

# Optimize to get the best combination of continuous and categorical variables
best_cont, best_cat = optimizer.optimize(new_cont, new_cat, n_predictions=5)
print(f"\nBest continuous: {best_cont}, Best categorical: {best_cat}")


Prediction 1: Best continuous: [0.6], Best categorical: omic2, UCB value: 0.8510456065515916, Reward: 0.27762397639061587
Prediction 2: Best continuous: [0.3], Best categorical: omic1, UCB value: 1.90326688006542, Reward: 0.5637804369378767
Prediction 3: Best continuous: [0.6], Best categorical: omic1, UCB value: 1.3655049452101133, Reward: 0.5558082584400276
Prediction 4: Best continuous: [0.6], Best categorical: omic4, UCB value: 2.017893659580081, Reward: 0.7021030569613053
Prediction 5: Best continuous: [0.6], Best categorical: omic1, UCB value: 2.022913458382277, Reward: 0.383824373195297

Predictions with UCB values:
Best continuous: [0.6], Best categorical: omic2, UCB value: 0.8510456065515916
Best continuous: [0.3], Best categorical: omic1, UCB value: 1.90326688006542
Best continuous: [0.6], Best categorical: omic1, UCB value: 1.3655049452101133
Best continuous: [0.6], Best categorical: omic4, UCB value: 2.017893659580081
Best continuous: [0.6], Best categorical: omic1, UCB val